In [7]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT,COMPLEX_MOVEMENT,RIGHT_ONLY
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecTransposeImage
from matplotlib import pyplot as plt
import random
import cv2
import numpy as np

In [8]:
def allWorldStages():
    worlds = [1]
    #Training on Stages 1-3 for each world, testing on final stage of each world
    stages = [1]

    for world in worlds:
        for stage in stages:
            environment_name = f"SuperMarioBros-{world}-{stage}-v0"
            yield environment_name

In [9]:
env_names = [name for name in allWorldStages()]
print(env_names)

['SuperMarioBros-1-1-v0']


In [10]:
def make_env(env_name):
    env = gym_super_mario_bros.make(env_name)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    env = GrayScaleObservation(env, keep_dim=True)
    return env

In [11]:
env_names = [name for name in allWorldStages()]
env_list = [make_env(name) for name in env_names]

env = DummyVecEnv([lambda env_name=name: make_env(env_name) for name in env_names])

env = VecFrameStack(env, 4, channels_order='last')
env = VecTransposeImage(env)
state = env.reset()
print(state.shape)

(1, 4, 240, 256)


In [12]:
for _ in range(1000):
    obs = env.render(mode='human')
    actions = [env.action_space.sample() for _ in range(env.num_envs)]
    obs, rewards, dones, info = env.step(actions)

env.close()